In [60]:
import random
import math
import json

In [61]:
def calculate_tour_cost(tour, dist_matrix):
    cost = 0
    n = len(tour)
    for i in range(n):
        curr_city = tour[i]
        next_city = tour[(i + 1) % n]
        cost += dist_matrix[curr_city][next_city]
    return cost

In [62]:
def get_neighbor_tour(curr_tour, n):
    neighbor_tour = curr_tour[:]
    i, j = random.sample(range(n), 2)
    neighbor_tour[i], neighbor_tour[j] = neighbor_tour[j], neighbor_tour[i]
    return neighbor_tour

In [63]:
def acceptance_probability(curr_cost, neighbor_cost, temp):
    if temp == 0:
        return 0.0
    if neighbor_cost < curr_cost:
        return 1.0
    return math.exp((curr_cost - neighbor_cost) / temp)

In [64]:
def solve_tsp_simulated_annealing(dist_matrix, start_city, init_temp, cooling_rate, iterations, output_file="tsp_intermediate_output.txt"):
    n = len(dist_matrix)

    # Initial tour (Nearest Neighbor heuristic)
    curr_tour = [start_city]
    unvisited = list(range(n))
    unvisited.remove(start_city)

    curr_city = start_city
    while unvisited:
        nearest_city = None
        min_dist = float('inf')
        for city in unvisited:
            if dist_matrix[curr_city][city] < min_dist:
                min_dist = dist_matrix[curr_city][city]
                nearest_city = city
        curr_tour.append(nearest_city)
        unvisited.remove(nearest_city)
        curr_city = nearest_city

    curr_cost = calculate_tour_cost(curr_tour, dist_matrix)

    best_tour = curr_tour[:]
    best_cost = curr_cost

    temp = init_temp

    # Simulated Annealing loop
    with open(output_file, "w") as f:
        for i in range(iterations):
            neighbor_tour = get_neighbor_tour(curr_tour, n)
            neighbor_cost = calculate_tour_cost(neighbor_tour, dist_matrix)

            prob = acceptance_probability(curr_cost, neighbor_cost, temp)
            rand_num = random.random()

            if prob > rand_num or neighbor_cost < curr_cost:
                curr_tour = neighbor_tour[:]
                curr_cost = neighbor_cost

            if curr_cost < best_cost:
                best_tour = curr_tour[:]
                best_cost = curr_cost

            temp *= cooling_rate

            f.write(f"Iteration: {i+1}, Temp: {temp:.4f}, Current Tour: {curr_tour}, Current Cost: {curr_cost:.2f}\n")

    return best_tour, best_cost

In [65]:
def main():
    # Load input data from file
    with open("tsp_input.txt", "r") as f:
        lines = f.readlines()

    # Parse distance matrix
    dist_matrix = []
    i = 0
    while True:
        row = list(map(int, lines[i].split()))
        dist_matrix.append(row)
        if len(row) != len(dist_matrix[0]): # Checking if it's the last row of the matrix
            dist_matrix.pop() # Removing the last row
            break
        i += 1

    start_city = int(lines[i].strip())
    init_temp = float(lines[i+1].strip())
    cooling_rate = float(lines[i+2].strip())
    iterations = int(lines[i+3].strip())

    # Solving TSP using Simulated Annealing
    best_tour, best_cost = solve_tsp_simulated_annealing(dist_matrix, start_city, init_temp, cooling_rate, iterations)

    with open("tsp_output.txt", "w") as f:
        f.write(f"Best Tour: {best_tour}\n")
        f.write(f"Best Cost: {best_cost:.2f}\n")

if __name__ == "__main__":
    main()